# FIT5196 Task 2 in Assessment 1
#### Student Name: LICUN LIU
#### Student ID: 30901235

Date: 31/08/2020

Version: 1.0

Environment: Python 2.7.11 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* pandas (for dataframe, included in Anaconda Python 2.7) 
* re (for regular expression, included in Anaconda Python 2.7) 
* numpy (for numpy array, included in Anaconda Python 2.7) 

<font size=3, color="red"> Note: this is a sample notebook only. You will need to fill in the proper markdown and code blocks. You might also want to make necessary changes to the structure to meet your own needs. It is important to make sure the logic of how you finish the assessment is clearly shown in this notebook! </font>

## 1.  Import libraries 

In [2]:
# Code to import libraries as you need in this assessment, e.g.,
! pip install langid

import xlrd
import pandas as pd
import numpy as np
import os
import re
import langid
import nltk 
import matplotlib.pyplot as plt

from nltk.corpus import reuters
from nltk.tokenize import RegexpTokenizer

## 2. Parse Excel files with Pandas

In this section, you can write your python scripts to parse the correspondiing file.
You should 
* write proper notes for all code block in this notebook using the Markdown cells
* provide proper comment in your scripts
* run all cells to make sure scripts are runable. If the scripts cannot be run by the assessors, they will not be assessed and zero mark will be given to the task.

In [3]:
excel_data = pd.ExcelFile('30901235.xlsx')

## 3. Create a stop word list and read all words in the file
Stop word list: Context-dependent(more than 60 days and rare tokens)
All word list: Read all the documents and get a list of all the words

### Read all data in the .xlsx and save them as a dataframe

In [4]:
# Create some variables
stop_data = pd.DataFrame(columns = ["text","id","created_at"])
new_stop_list = []
word_appear  = {}
all_words = []


# Read the content from all excel forms
for data in excel_data.sheet_names:
    # Using stop_df list to save the content
    stop_df = excel_data.parse(data)
    stop_df = stop_df.dropna(axis=1, how='all')
    stop_df = stop_df.dropna(axis=0, how='all')
    stop_df.columns = ["text","id","created_at"]
    stop_df = stop_df.drop(stop_df.index[0])
    stop_df.index = range(len(stop_df.index))

    # Only keeps the tweets that are in English language and Reset the row indices
    stop_df_index = []
    for index in range(len(stop_df.index)):
        if langid.classify(str(stop_df.loc[index, 'text']))[0] != 'en':
            stop_df_index.append(index)
    for i in stop_df_index:
        stop_df.drop(i,0,inplace = True)
    stop_df.index = range(len(stop_df.index))
    
 

    # Get the word token of all the content
    i=0
    from nltk.tokenize import RegexpTokenizer 
    tokenizer = RegexpTokenizer(r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?")
    unigram_tokens = []
    
    while i < len(stop_df.index):
        raw_text = str(stop_df.loc[i, 'text']).lower()
        unigram_tokens =  unigram_tokens + tokenizer.tokenize(raw_text) 
        # Using a space to separate the words in different tweets
        unigram_tokens.append(" ")
        i = i+1
    
    unique_stem_list = list(set(unigram_tokens))
    
    # Using all_words list to save all word tokens
    all_words = all_words + unigram_tokens
    
    # Using word_appear dictionary to count the number of occurrences of each word token
    for i in unique_stem_list:
        if i in word_appear:
            word_appear[i] = word_appear[i] + 1
        else:
            word_appear[i] = 1
        
        # Save the tokens with the length less than 3 in the stop list
        if len(i)<3:
            new_stop_list.append(i)

new_stop = new_stop_list

### Add the context-dependent and rare tokens in the stop list.

In [5]:
# Using for loop to find context-dependent and rare tokens
for i in word_appear:
    if (word_appear[i] < 5) or (word_appear[i] > 60):
        new_stop_list.append(i)

# Save in the stop list
new_stop_list = list(set(new_stop_list))
new_stop_list.sort()  


## 4. Using PMI measure to find first 200 meaningful bigrams

In [6]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(all_words)
abc = finder.nbest(bigram_measures.pmi, 200)

### Save first 200 meaningful bigrams in a list

In [7]:
bli_list = []
i=0
while i < len(abc):
    bli_list.append(abc[i][0]+'_'+abc[i][1])
    i = i +1
    

### Stop Words Removal
Using stiowirds_en.txt and the stop_word list to remove the stop words

In [8]:
import nltk

stopwords_list_570 = []
with open('./stopwords_en.txt') as f:
    stopwords_list_570 = f.read().splitlines()

stopped_tokens = [w for w in all_words if w not in stopwords_list_570]
stopped_tokens = [w for w in stopped_tokens if w not in new_stop]


### Tokens should be stemmed using the Porter stemmer

In [9]:
from nltk.stem import PorterStemmer
porter = PorterStemmer()
stem_list = []
for i in stopped_tokens:
    word = porter.stem(i)
    stem_list.append(word)



### Put stemmed tokens and first 200 meaningful bigrams together

In [10]:
stem_list = stem_list + bli_list
unique_stem_list = list(set(stem_list))
unique_stem_list.sort()

## 5. Output the answer of 30901235_vocab in a txt file

In [11]:
nlist = list(range(0,len(unique_stem_list)))
dic = dict(zip(unique_stem_list,nlist))

fw = open("30901235_vocab.txt",'w+')

count = 0
for i in dic:   
    fw.write(i + ":"+ str(count) + '\n')
    count =count + 1
fw.close()

## 6. Create countVec, 100uni and bi_100

In [12]:
df = pd.DataFrame(columns = ["text","id","created_at"]) 
for data in excel_data.sheet_names:
    
    ### Drop NULL columns and rows
    ### Remove all the rows and columns that are empty, i.e., only contains NaNs.
    dframe = excel_data.parse(data)
    dframe = dframe.dropna(axis=1, how='all')
    dframe = dframe.dropna(axis=0, how='all')
    dframe.columns = ["text","id","created_at"]
    dframe = dframe.drop(dframe.index[0])
    df = dframe
    
    ### Reset the row indices with a list of integers.
    df.index = range(len(df.index))
    
    ### Only keeps the tweets that are in English language and Reset the row indices
    del_index = []
    for index in range(len(df.index)):
        if langid.classify(str(df.loc[index, 'text']))[0] != 'en':
            del_index.append(index)
    for index in del_index:
        df.drop(index,0,inplace = True)
    df.index = range(len(df.index))
    
    # Word Tokenization
    i=0
    from nltk.tokenize import RegexpTokenizer 
    tokenizer = RegexpTokenizer("[a-zA-Z]+(?:[-'][a-zA-Z]+)?")
    unigram_tokens = []

    while i < len(df.index):
        raw_text = str(df.loc[i, 'text']).lower()
        unigram_tokens =  unigram_tokens + tokenizer.tokenize(raw_text) 
        i = i+1
    

    # Create top 100 most frequent bigrams
    from nltk.probability import *
    from nltk.util import ngrams
    bigrams = ngrams(unigram_tokens, n = 2)
    fdbigram = FreqDist(bigrams)
    
    daily_bigram = fdbigram.most_common(100) 
    
    # Save top 100 most frequent bigrams in 30901235_100bi
    fi = open("30901235_100bi.txt", 'a+')
    fi.write(data + ':')

    fi.write(str(daily_bigram))
    fi.write('\n')
    fi.close()
    
    ## Stop Words Removal
    ### Using stiowirds_en.txt and my stop list to remove the stop words
    import nltk
    stopwords_list_570 = []
    with open('./stopwords_en.txt') as f:
        stopwords_list_570 = f.read().splitlines()

    stopped_tokens = [w for w in unigram_tokens if w not in stopwords_list_570]
    
    stopped_token = [w for w in stopped_tokens if w not in new_stop_list]
    
    ## Tokens should be stemmed using the Porter stemmer
    from nltk.stem import PorterStemmer
    porter = PorterStemmer()
    stem_list = []
    for i in stopped_token:
        word = porter.stem(i)
        stem_list.append(word)
    
    ## Creating Count Vectors
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer(analyzer = "word") 
    
    from nltk.probability import *
    fd_1 = FreqDist(stem_list)
    most_common = fd_1.most_common()
    
    ### Output the answer of count_vec in a txt file
    files = open("30901235_countVec.txt",'a+')
    files.write(data + ',')
    for i in most_common:
        if i[0] in dic:
            files.write(str(dic[i[0]]) + ":" + str(i[1]) + ",")
    files.write('\n')   
    files.close()
    
    
    # Create first 100 most comman unigrams
    most_common_100 = fd_1.most_common(100)
    
    # Save top 100 most frequent unigrams in 30901235_100uni
    fil = open("30901235_100uni.txt", 'a+')
    fil.write(data + ':')

    fil.write(str(most_common_100))
    fil.write('\n')
    fil.close()
    
